In [1]:
import coverage_v1 as cov
import geopandas as gpd
import plotly.express as px

In [2]:
tles = cov.gen_sats(sat_nos=[39084, 49260])
times = cov.gen_times(start_yr=2021, start_mo=11, start_day=27, days=1, step_min=1)
df1 = cov.prop_orbits(tles, times)
df1 = cov.add_bearing(df1)
dt_df = df1.set_index("time")


Satellite(s) Loaded from TLE:
[<EarthSatellite LANDSAT 8 catalog #39084 epoch 2021-12-04 09:40:39 UTC>]
[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2021-12-04 10:30:08 UTC>]
Propogation time: 
 2021-11-27 00:00:00+00:00 
to 
 2021-11-27 23:59:00+00:00
Adding LANDSAT 8
Adding LANDSAT 9


In [3]:
# fig1 = px.scatter_geo(dt_df, lon="lon", lat="lat", color="satellite", projection="orthographic")
# fig1.show()

In [4]:
df1

,time,satellite,lat,lon,altitude,bearing,sunlit
0,2021-11-27 00:00:00+00:00,LANDSAT 8,67.361484,-46.940843,713.451943,337.183481,False
1,2021-11-27 00:01:00+00:00,LANDSAT 8,70.692382,-51.243374,713.971113,333.633106,False
2,2021-11-27 00:02:00+00:00,LANDSAT 8,73.897486,-57.119595,714.403823,328.510281,False
3,2021-11-27 00:03:00+00:00,LANDSAT 8,76.885552,-65.580537,714.741203,320.803556,False
4,2021-11-27 00:04:00+00:00,LANDSAT 8,79.471456,-78.347546,714.976298,308.791132,True
...,...,...,...,...,...,...,...
2875,2021-11-27 23:55:00+00:00,LANDSAT 9,70.926233,-50.119351,714.536840,333.313458,False
2876,2021-11-27 23:56:00+00:00,LANDSAT 9,74.118649,-56.141681,714.943766,328.044700,False
2877,2021-11-27 23:57:00+00:00,LANDSAT 9,77.084647,-64.844877,715.252628,320.095865,False
2878,2021-11-27 23:58:00+00:00,LANDSAT 9,79.629397,-78.004318,715.456701,307.691121,True


In [5]:
# inst = cov.gen_instrument(name="instrument", fl=100, pitch=0.017, h_pix=1024*3, v_pix=768, mm=True)
inst = cov.gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True)

inst

{'name': 'tirs',
 'fl': 178,
 'pitch': 0.025,
 'h_pix': 1850,
 'v_pix': 1800,
 'mm': True,
 'half_diag_deg': 10.385591925260362,
 'az1': 45.78482460299189,
 'az2': 314.2151753970081}

In [6]:
df1

,time,satellite,lat,lon,altitude,bearing,sunlit
0,2021-11-27 00:00:00+00:00,LANDSAT 8,67.361484,-46.940843,713.451943,337.183481,False
1,2021-11-27 00:01:00+00:00,LANDSAT 8,70.692382,-51.243374,713.971113,333.633106,False
2,2021-11-27 00:02:00+00:00,LANDSAT 8,73.897486,-57.119595,714.403823,328.510281,False
3,2021-11-27 00:03:00+00:00,LANDSAT 8,76.885552,-65.580537,714.741203,320.803556,False
4,2021-11-27 00:04:00+00:00,LANDSAT 8,79.471456,-78.347546,714.976298,308.791132,True
...,...,...,...,...,...,...,...
2875,2021-11-27 23:55:00+00:00,LANDSAT 9,70.926233,-50.119351,714.536840,333.313458,False
2876,2021-11-27 23:56:00+00:00,LANDSAT 9,74.118649,-56.141681,714.943766,328.044700,False
2877,2021-11-27 23:57:00+00:00,LANDSAT 9,77.084647,-64.844877,715.252628,320.095865,False
2878,2021-11-27 23:58:00+00:00,LANDSAT 9,79.629397,-78.004318,715.456701,307.691121,True


In [7]:
# df = df1[df1.satellite == "LANDSAT 8"]
df = df1
df.loc[0]

time         2021-11-27 00:00:00+00:00
satellite                    LANDSAT 8
lat                          67.361484
lon                         -46.940843
altitude                    713.451943
bearing                     337.183481
sunlit                           False
Name: 0, dtype: object

In [8]:
df1 = df1.dropna()

In [9]:
df, polys = cov.gen_los_offsets(df1, inst)

In [10]:
cmap = px.colors.qualitative.Plotly
sats = polys.satellite.unique()
color_key = dict(zip(sats, cmap))

color_key


polys['color'] = polys.apply(
    lambda row: color_key[row.satellite], axis=1)



In [11]:
polys = polys.to_crs("ESRI:54009")
polys = polys[polys.area < polys.area.quantile(0.98)]
polys

,geometry,satellite,color
0,"POLYGON ((-2432444.235 7642536.657, -2703035.3...",LANDSAT 8,#636EFA
1,"POLYGON ((-2395918.112 7921971.679, -2689828.8...",LANDSAT 8,#636EFA
2,"POLYGON ((-2371415.570 8175212.935, -2696453.8...",LANDSAT 8,#636EFA
3,"POLYGON ((-2379980.864 8395128.282, -2747392.7...",LANDSAT 8,#636EFA
4,"POLYGON ((-2472639.314 8569268.403, -2894494.6...",LANDSAT 8,#636EFA
...,...,...,...
2873,"POLYGON ((-2351061.948 7663412.830, -2621811.0...",LANDSAT 9,#EF553B
2874,"POLYGON ((-2323053.534 7941067.780, -2617207.8...",LANDSAT 9,#EF553B
2875,"POLYGON ((-2308479.383 8192130.761, -2633896.3...",LANDSAT 9,#EF553B
2876,"POLYGON ((-2329469.040 8409198.903, -2697365.9...",LANDSAT 9,#EF553B


In [12]:
# fig = polys.explore(color="color", tooltip="satellite")
polys.explore(color="color", tooltip="satellite")
# for tomorrow... https://towardsdatascience.com/around-the-world-in-80-lines-crossing-the-antimeridian-with-python-and-shapely-c87c9b6e1513

In [13]:
# fig.save("v1_map.html")

In [14]:
polys.area.mean()

34401878493.4471

In [15]:
polys.to_parquet("v1_test.parquet")

/var/folders/fc/b2n8k5vx0fd5pss09qxrjkd80000gn/T/ipykernel_29684/3757393790.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  polys.to_parquet("v1_test.parquet")
